In [1]:
import pandas as pd
import re

In [2]:
continents = ["europe", "asia", "america"]

In [37]:
dataframes = {}
for continent in continents:
    df = pd.read_parquet(f'data/{continent}_data.parquet')
    df_copy = df.copy()
    df_copy.drop("header_line", axis=1, inplace=True)

    stepIDs = []
    action_modes = []
    header_lines = df["header_line"].values
    for header in header_lines:
        match = re.search(r"\[(.*?)\]", header)
        if match:
            result = match.group(1)
            stepIDs.append(result)
        else:
            stepIDs.append(None)
        
        
        last_ten_characters = header[-10:]
        if "Received" in header:
            action_modes.append("Received")
        elif "Saved" in header:
            action_modes.append("Saved")
        else:
            action_modes.append(None)

    df_copy["stepID"] = stepIDs
    df_copy["action_mode"] = action_modes
    
    dataframes[continent] = df_copy
    

In [3]:
europe = pd.read_parquet('data/europe_data.parquet')
actions = pd.read_excel('data/Liste der Actions.xlsx')
action_names = actions["Action Name"].values

In [29]:
filtered_df = None
for action in action_names:
    tempdf = None
    if action == action_names[0]:
        tempDF = europe[europe["action_name"] == action]
        tempDF = tempDF.head(10)
    else:
        actiondf = europe[europe["action_name"] == action].head(10)
        tempDF = pd.concat([filtered_df, actiondf])
    filtered_df = tempDF


In [34]:
filtered_df.to_csv('data/actions.csv')